# Code to get the hospital closest to the centre of an area unit, and area unit of hospital

We have a hospital dataframe, but don't know the area unit of the hospital. In this notebook we will take the hospital dataframe and the area unit dataframes and use the lat and long coordinates that each have to calculate the nearest hospital to certain unit area

In [5]:
# Installing  and attaching packages
using Pkg 
#Pkg.add("DataFrames")
#Pkg.add("CSV")

using CSV, DataFrames

First we need to load the datasets

In [6]:
#Loading datasets
area_units = CSV.read("areaXY.csv", DataFrame)
hospitals = CSV.read("CanterburyHospitalsLocationsFinal.csv", DataFrame)


,Premises.Name,Certification.Service.Type,Service.Types
,String,String,String
1,Ashburton Hospital,Public Hospital,"Medical, Maternity"
2,Burwood Hospital,Public Hospital,"Surgical, Geriatric, Psychogeriatric, Medical"
3,Chatham Island Health Centre,Public Hospital,Medical
4,Christchurch Hospital,Public Hospital,"Childrens health, Medical, Surgical, Maternity"
5,Darfield Hospital,Public Hospital,"Medical, Geriatric"
6,Ellesmere Hospital,Public Hospital,"Medical, Geriatric"
7,Hillmorton Hospital,Public Hospital,Mental health
8,Kaikoura Hospital,Public Hospital,"Medical, Maternity, Geriatric"
9,Lincoln Maternity Hospital,Public Hospital,Maternity


Note that area units has latitude and longitude switched around the wrong way, so we clean up this table.

In [8]:
# Fixing column names of area units table
area_units = select(area_units, "AU2017_NAME", "Longitude" => "Latitude", "Latitude" => "Longitude")

,AU2017_NAME,Latitude,Longitude
,String,Float64,Float64
1,Addington,-43.5435,172.62
2,Aidanfield,-43.5644,172.569
3,Akaroa,-43.8067,172.966
4,Akaroa Harbour,-43.7716,172.939
5,Allenton East,-43.8922,171.753
6,Allenton West,-43.8905,171.742
7,Amberley,-43.1558,172.73
8,Amuri,-42.5893,172.72
9,Aorangi,-43.4993,172.595


In [9]:
# Crossjoin hospitals and area units to get one large df with each area unit matched with each hospital
cross_df = crossjoin(hospitals, area_units, makeunique = true)
describe(cross_df)

,variable,mean,min
,Symbol,Union…,Any
1,Premises.Name,,Ashburton Hospital
2,Certification.Service.Type,,NGO Hospital
3,Service.Types,,"Childrens health, Maternity, Surgical, Medical, Mental health"
4,Total.Beds,87.0476,3
5,Premises.Address,,1 Lincoln Road
6,Premises.Address.Suburb.Road,,Allenton
7,Premises.Address.Town.City,,Ashburton
8,Premises.Address.Post.Code,7792.24,7300
9,DHB.Name,,Canterbury District Health Board


In [10]:
# Change column names to improve them
cross_df = rename(cross_df, 
    "Premises.Name" => "Hospital_Name",
    "Certification.Service.Type" => "Certification_Service_Type",
    "Service.Types" => "Service_Types",
    "Total.Beds" => "Total_Beds",
    "Premises.Address" => "Hospital_Address",
    "Premises.Address.Suburb.Road" => "Hospital_Suburb",
    "Premises.Address.Town.City" => "Hospital_City",
    "Premises.Address.Post.Code" => "Hospital_Postcode",
    "DHB.Name" => "DHB_Name",
    "Latitude" => "Hospital_Lat",
    "Longitude" => "Hospital_Long",
    "Latitude_1" => "AU_Lat",
    "Longitude_1" => "AU_Long",
    "Childrens health" => "Childrens_Health",
    "Dementia care" => "Dementia_Care",
    "Mental health" => "Mental_Health"
    )


,Hospital_Name,Certification_Service_Type,Service_Types,Total_Beds
,String,String,String,Int64
1,Ashburton Hospital,Public Hospital,"Medical, Maternity",56
2,Ashburton Hospital,Public Hospital,"Medical, Maternity",56
3,Ashburton Hospital,Public Hospital,"Medical, Maternity",56
4,Ashburton Hospital,Public Hospital,"Medical, Maternity",56
5,Ashburton Hospital,Public Hospital,"Medical, Maternity",56
6,Ashburton Hospital,Public Hospital,"Medical, Maternity",56
7,Ashburton Hospital,Public Hospital,"Medical, Maternity",56
8,Ashburton Hospital,Public Hospital,"Medical, Maternity",56
9,Ashburton Hospital,Public Hospital,"Medical, Maternity",56


Now we can write a function that will take the lat long of the hospitals and area units, and calculate the distance between them in metres - we then apply the function to the cross joined dataframe lat and long columns 

In [11]:
## Function to convert degrees to radians since we use radians to calculate distance
function deg2rad(x)

    return x * pi / 180
end

# Function to calculate distance in metres from two lat and long coordinates
function spherical_distance(lat1, long1, lat2, long2)
   
   
    x1 = 0.5*pi - deg2rad(lat1) # creating a variable for x1 in metres, converting latitudes of point 1 and 2 into radians
    x2 = 0.5*pi - deg2rad(lat2)
   
    r = 0.5*(6378.137 + 6356.752) # mean radius of the Earth in kilometers
     
    # mathematical function that takes our inputs and calculates distance accounting for points being on a sphere
    t = sin(x1)*sin(x2)*cos(deg2rad(long1)-deg2rad(long2)) + cos(x1)*cos(x2)
    return float(r * acos(t))
end

# Testing function
spherical_distance(-43.57148, 172.61959, -43.5429307391304, 172.614197202899)

3.2023500502926416

Now that we have all the distances from each hospital to each suburb, we can group the dataframe by area unit and return the mimumim value fo the distances columns, to see which hospital is closesest to centre point of that area unit. Credit to user Bogumił Kamiński on stack overflow post https://stackoverflow.com/questions/65024962/select-rows-of-a-dataframe-containing-minimum-of-grouping-variable-in-julia for the following line of code.

In [12]:
# Apply function row-wise to 2 separate sets of lat and long columns in the dataframe
distance_df = transform!(cross_df, [:Hospital_Lat, :Hospital_Long, :AU_Lat, :AU_Long] => ByRow(spherical_distance) => :Distance )

,Hospital_Name,Certification_Service_Type,Service_Types,Total_Beds
,String,String,String,Int64
1,Ashburton Hospital,Public Hospital,"Medical, Maternity",56
2,Ashburton Hospital,Public Hospital,"Medical, Maternity",56
3,Ashburton Hospital,Public Hospital,"Medical, Maternity",56
4,Ashburton Hospital,Public Hospital,"Medical, Maternity",56
5,Ashburton Hospital,Public Hospital,"Medical, Maternity",56
6,Ashburton Hospital,Public Hospital,"Medical, Maternity",56
7,Ashburton Hospital,Public Hospital,"Medical, Maternity",56
8,Ashburton Hospital,Public Hospital,"Medical, Maternity",56
9,Ashburton Hospital,Public Hospital,"Medical, Maternity",56


In [13]:
# Show selected columns
distance_df[!, [:Hospital_Name, :AU2017_NAME, :Distance]]

,Hospital_Name,AU2017_NAME,Distance
,String,String,Float64
1,Ashburton Hospital,Addington,80.3395
2,Ashburton Hospital,Aidanfield,75.6005
3,Ashburton Hospital,Akaroa,98.2812
4,Ashburton Hospital,Akaroa Harbour,96.6448
5,Ashburton Hospital,Allenton East,0.61726
6,Ashburton Hospital,Allenton West,0.55411
7,Ashburton Hospital,Amberley,114.134
8,Ashburton Hospital,Amuri,165.098
9,Ashburton Hospital,Aorangi,81.1279


We want to find which hospital for each type is closest to each area unit. Using our one hot encoded columns, we find the closest hospital separately for each type, and the join the results.

In [14]:
# create 8 different dataframes for each type that we will eventually groupby by each and join merge

# Medical
medical_df = select(distance_df[(distance_df[!, :Medical] .== 1),:],
                    :AU2017_NAME,
                    "Hospital_Name" => "Closest_Medical_Hospital",
                    "Distance" => "Distance_Medical_Hospital"
                    )
closest_medical = combine(medical_df -> filter(:Distance_Medical_Hospital => ==(minimum(medical_df.Distance_Medical_Hospital)), medical_df), groupby(medical_df, :AU2017_NAME))

# Childrens health
childrens_health_df = select(distance_df[(distance_df[!, :Childrens_Health] .== 1),:],
                    :AU2017_NAME,
                    "Hospital_Name" => "Closest_Childrens_Health_Hospital",
                    "Distance" => "Distance_Childrens_Health_Hospital"
                    )
closest_childrens_health = combine(childrens_health_df -> filter(:Distance_Childrens_Health_Hospital => ==(minimum(childrens_health_df.Distance_Childrens_Health_Hospital)), childrens_health_df), groupby(childrens_health_df, :AU2017_NAME))

# Dementia care
dementia_care_df = select(distance_df[(distance_df[!, :Dementia_Care] .== 1),:],
                    :AU2017_NAME,
                    "Hospital_Name" => "Closest_Dementia_Care_Hospital",
                    "Distance" => "Distance_Dementia_Care_Hospital"
                    )
closest_dementia_care = combine(dementia_care_df -> filter(:Distance_Dementia_Care_Hospital => ==(minimum(dementia_care_df.Distance_Dementia_Care_Hospital)), dementia_care_df), groupby(dementia_care_df, :AU2017_NAME))

# Geriatric
geriatric_df = select(distance_df[(distance_df[!, :Geriatric] .== 1),:],
                    :AU2017_NAME,
                    "Hospital_Name" => "Closest_Geriatric_Hospital",
                    "Distance" => "Distance_Geriatric_Hospital"
                    )
closest_geriatric = combine(geriatric_df -> filter(:Distance_Geriatric_Hospital => ==(minimum(geriatric_df.Distance_Geriatric_Hospital)), geriatric_df), groupby(geriatric_df, :AU2017_NAME))

# Maternity
maternity_df = select(distance_df[(distance_df[!, :Maternity] .== 1),:],
                    :AU2017_NAME,
                    "Hospital_Name" => "Closest_Maternity_Hospital",
                    "Distance" => "Distance_Maternity_Hospital"
                    )
closest_maternity = combine(maternity_df -> filter(:Distance_Maternity_Hospital => ==(minimum(maternity_df.Distance_Maternity_Hospital)), maternity_df), groupby(maternity_df, :AU2017_NAME))

# Mental health
mental_health_df = select(distance_df[(distance_df[!, :Mental_Health] .== 1),:],
                    :AU2017_NAME,
                    "Hospital_Name" => "Closest_Mental_Health_Hospital",
                    "Distance" => "Distance_Mental_Health_Hospital"
                    )
closest_mental_health = combine(mental_health_df -> filter(:Distance_Mental_Health_Hospital => ==(minimum(mental_health_df.Distance_Mental_Health_Hospital)), mental_health_df), groupby(mental_health_df, :AU2017_NAME))

# Psychogeriatric
psychogeriatric_df = select(distance_df[(distance_df[!, :Psychogeriatric] .== 1),:],
                    :AU2017_NAME,
                    "Hospital_Name" => "Closest_Psychogeriatric_Hospital",
                    "Distance" => "Distance_Psychogeriatric_Hospital"
                    )
closest_psychogeriatric = combine(psychogeriatric_df -> filter(:Distance_Psychogeriatric_Hospital => ==(minimum(psychogeriatric_df.Distance_Psychogeriatric_Hospital)), psychogeriatric_df), groupby(psychogeriatric_df, :AU2017_NAME))

# Surgical
surgical_df = select(distance_df[(distance_df[!, :Surgical] .== 1),:],
                    :AU2017_NAME,
                    "Hospital_Name" => "Closest_Surgical_Hospital",
                    "Distance" => "Distance_Surgical_Hospital"
                    )
closest_surgical = combine(surgical_df -> filter(:Distance_Surgical_Hospital => ==(minimum(surgical_df.Distance_Surgical_Hospital)), surgical_df), groupby(surgical_df, :AU2017_NAME))

,AU2017_NAME,Closest_Surgical_Hospital,Distance_Surgical_Hospital
,String,String,Float64
1,Addington,Christchurch Hospital,1.11558
2,Aidanfield,Christchurch Hospital,5.6341
3,Akaroa,Christchurch Hospital,40.8662
4,Akaroa Harbour,Forte Health Hospital,36.5193
5,Allenton East,Bidwill Trust Hospital,69.0885
6,Allenton West,Bidwill Trust Hospital,68.7317
7,Amberley,Burwood Hospital,36.4268
8,Amuri,Burwood Hospital,99.2386
9,Aorangi,St George's Hospital,2.30784


In [15]:
# Joining all closest hopsitals on AU
closest_hospitals_df = innerjoin(closest_medical,
    closest_childrens_health,
    closest_dementia_care,
    closest_geriatric,
    closest_maternity,
    closest_mental_health,
    closest_psychogeriatric,
    closest_surgical,
    on = :AU2017_NAME)

,AU2017_NAME,Closest_Medical_Hospital,Distance_Medical_Hospital
,String,String,Float64
1,Addington,Christchurch Hospital,1.11558
2,Aidanfield,Christchurch Hospital,5.6341
3,Akaroa,Christchurch Hospital,40.8662
4,Akaroa Harbour,Christchurch Hospital,36.5427
5,Allenton East,Ashburton Hospital,0.61726
6,Allenton West,Ashburton Hospital,0.55411
7,Amberley,Rangiora Hospital,18.5311
8,Amuri,Waikari Hospital,42.1628
9,Aorangi,Nurse Maude Hospice,2.57961


In [16]:
# Testing, show selected columns
closest_hospitals_df[!, [:AU2017_NAME, :Closest_Medical_Hospital, :Closest_Surgical_Hospital]]

,AU2017_NAME,Closest_Medical_Hospital,Closest_Surgical_Hospital
,String,String,String
1,Addington,Christchurch Hospital,Christchurch Hospital
2,Aidanfield,Christchurch Hospital,Christchurch Hospital
3,Akaroa,Christchurch Hospital,Christchurch Hospital
4,Akaroa Harbour,Christchurch Hospital,Forte Health Hospital
5,Allenton East,Ashburton Hospital,Bidwill Trust Hospital
6,Allenton West,Ashburton Hospital,Bidwill Trust Hospital
7,Amberley,Rangiora Hospital,Burwood Hospital
8,Amuri,Waikari Hospital,Burwood Hospital
9,Aorangi,Nurse Maude Hospice,St George's Hospital


Now we can find what area unit a hospital is in, by grouping by hospital and finding the smallest distance from hospital to any area unit.
NB: this is not as precise as checking if hospital in inside a certain boundary, as it is possible for a hospital to be in a certain Area Unit and still be closer to center of a different area unit. However we with the tools at our disposal, we are happy with this result. 

In [22]:
# Finding closest centre of area unit for each hospital
hospitals_area_unit_df = combine(cross_df -> filter(:Distance => ==(minimum(cross_df.Distance)), cross_df), groupby(cross_df, :Hospital_Name))

# Removing columns and renaming one
hospitals_area_unit_df =  select!(hospitals_area_unit_df, Not(:Distance))
hospitals_area_unit_df =  select!(hospitals_area_unit_df, Not(:AU_Lat))
hospitals_area_unit_df =  select!(hospitals_area_unit_df, Not(:AU_Long))
hospitals_area_unit_df = rename(hospitals_area_unit_df, 
    "AU2017_NAME" => "Hospital_Area_Unit" 
    )


describe(hospitals_area_unit_df)

,variable,mean,min
,Symbol,Union…,Any
1,Hospital_Name,,Ashburton Hospital
2,Certification_Service_Type,,NGO Hospital
3,Service_Types,,"Childrens health, Maternity, Surgical, Medical, Mental health"
4,Total_Beds,87.0476,3
5,Hospital_Address,,1 Lincoln Road
6,Hospital_Suburb,,Allenton
7,Hospital_City,,Ashburton
8,Hospital_Postcode,7792.24,7300
9,DHB_Name,,Canterbury District Health Board


Now we have our dataframes, we can save them

In [24]:
# Writing df to csv file
CSV.write("Closest Hospitals.csv", closest_hospitals_df)
CSV.write("Hospitals.csv", hospitals_area_unit_df)

"Hospitals.csv"